## Зависимости

In [1]:
!pip install diffusers invisible_watermark transformers accelerate safetensors fastapi nest-asyncio pyngrok uvicorn nest-asyncio python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00


## Код сервера

In [ ]:
import io
from enum import Enum
from pathlib import Path
from queue import Queue
from threading import Thread
from typing import Annotated
from uuid import uuid4

import nest_asyncio
import PIL
import torch
import uvicorn
from diffusers import (
    StableDiffusionInstructPix2PixPipeline,
    StableDiffusionXLImg2ImgPipeline,
)
from fastapi import FastAPI, File, HTTPException, status
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import conf, ngrok
from transformers import pipeline


sd_xl_refiner_pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

sd_instruct_pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix",
    torch_dtype=torch.float16,
    safety_checker=None,
).to("cuda")

rmbg_pipe = pipeline(
    "image-segmentation",
    model="briaai/RMBG-1.4",
    revision="refs/pr/9",
    trust_remote_code=True,
)

tmp_dir = Path("tmp")
if not tmp_dir.is_dir():
    tmp_dir.mkdir()

static_dir = Path("static")
if not static_dir.is_dir():
    static_dir.mkdir()

MAX_PHOTO_BYTES = 1572864


class Model(Enum):
    sd_xl_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"
    sd_instruct = "timbrooks/instruct-pix2pix"
    rmbg = "briaai/RMBG-1.4"

    
class PhotoResponse(BaseModel):
    photo: str


class HTTPError(BaseModel):
    detail: str


def processing_photo_worker(tasks_queue: Queue) -> None:
    while True:
        task = tasks_queue.get()
        if task is None:
            break
        model, prompt, image, photo_name = task
        out_path = static_dir / photo_name
        if model == Model.sd_xl_refiner:
            processed_image = sd_xl_refiner_pipe(prompt, image=image).images[0]
            processed_image.save(out_path, "png")
        elif model == Model.sd_instruct:
            processed_image = sd_instruct_pipe(
                prompt, image=image, num_inference_steps=15
            ).images[0]
            processed_image.save(out_path, "png")
        elif model == Model.rmbg:
            tmp_photo_path = tmp_dir / photo_name
            image.save(tmp_photo_path, "png")
            rmbg_pipe(tmp_photo_path, out_name=out_path)
            tmp_photo_path.unlink()
        else:
            error_msg = f"Unknown {model} model"
            raise ValueError(error_msg)


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.mount("/photos", StaticFiles(directory=static_dir), name="static")

tasks_queue = Queue()


@app.post(
    "/",
    responses={
        400: {
            "model": HTTPError,
            "description": "Image is too big or file is not an image",
        },
    },
)
async def run_photo_processing(
    file: Annotated[bytes, File()], model: Model, prompt: str = ""
) -> PhotoResponse:
    if len(file) > MAX_PHOTO_BYTES:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Image is too big",
        )
    try:
        image = PIL.Image.open(io.BytesIO(file))
    except PIL.UnidentifiedImageError as error:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="File is not an image",
        ) from error
    photo_name = f"{uuid4().hex}.png"
    tasks_queue.put((model, prompt, image, photo_name))
    return PhotoResponse(photo=photo_name)


config = conf.PyngrokConfig(
    auth_token="2d98dg0EMgbt1t5dfHvwNjlC89h_7yU4eHniPiexTz4iST678",
    api_key="2d98ohi2s3tLh2XZtZzI7QuV10f_4hv6dBiz6bvwBXqouYQ2b",
)
ngrok_tunnel = ngrok.connect(8000, pyngrok_config=config)
print("Public URL:", ngrok_tunnel.public_url)

nest_asyncio.apply()
thread = Thread(target=processing_photo_worker, args=(tasks_queue,))
thread.start()
uvicorn.run(app, port=8000)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-03-09 23:50:59.358983: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 23:50:59.359081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 23:50:59.487654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model_index.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

INFO:     95.24.145.147:0 - "POST /?model=stabilityai%2Fstable-diffusion-xl-refiner-1.0&prompt=Make%20a%20barbie%20doll%20out%20of%20it.%20Keep%20your%20facial%20features%20the%20same%2C%20but%20make%20your%20hair%20blonde. HTTP/1.1" 200 OK


  0%|          | 0/15 [00:00<?, ?it/s]

INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     95.24.145.147:0 - "GET /7cf649dac02c4004b1d2c1b0bdf7e88d.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     95.24.145.147:0 - "GET /openapi.json HTTP/1.1" 200 

  0%|          | 0/15 [00:00<?, ?it/s]

INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa446526446669311b6d9d8bfd30f.png HTTP/1.1" 404 Not Found
INFO:     95.24.145.147:0 - "GET /62eaa44652644666